In [1]:
import torch
from torch import nn
from d2l import torch as d2l
from torch.optim import lr_scheduler
from torchvision import datasets,transforms
import os

In [2]:
class MyLeNet(nn.Module):
    def __init__(self):
        super(MyLeNet,self).__init__()
        
        self.Sigmoid = nn.Sigmoid()
        self.c1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,padding=2)
        self.s2 = nn.AvgPool2d(kernel_size=2,stride=2)     #池化层
        self.c3 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)
        self.s4 = nn.AvgPool2d(kernel_size=2,stride=2)     #池化层
        self.c5 = nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5)

        self.flatten = nn.Flatten()
        self.f6 = nn.Linear(120,84)
        self.output = nn.Linear(84,10)

    def forward(self, x): #x = 28 x 28 x 1
        x = self.c1(x)    #x = ((28+2x2))-(5-1)) x 28 x 6
        x = self.Sigmoid(x) #x = 28 x 28 x 6 [0,1]
        x = self.s2(x)      #x = ((28-2+2)/2) x 14 x 6
        x = self.c3(x)      #x = (14-4) x 10 x 16
        x = self.Sigmoid(x)
        x = self.s4(x)      #x = 5 x 5 x 16
        x = self.c5(x)      #x = 1 x 1 x 120
        x = self.flatten(x) #x = 120 x 1
        x = self.f6(x)      #x = 84 x 1
        x = self.output(x)  #x = 10 x 1

        return x

In [3]:
#转化数据格式
data_transform = transforms.Compose([transforms.ToTensor()])

#加载训练数据集
train_dataset = datasets.MNIST(root = "./data",train=True,transform=data_transform,download=True)
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size = 16,shuffle=True)
#加载测试数据集
test_dataset = datasets.MNIST(root = "./data",train=False,transform=data_transform,download=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = 16,shuffle=True)
# for x,u in enumerate(train_dataloader):
#     print(x)
#     print(u)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
model = MyLeNet().to(device)

#损失函数 交叉熵
lossfn = nn.CrossEntropyLoss()

#优化器
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01,momentum=0.9)

#学习率每隔10轮，变为原来的0.1
lr_scheduler = lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

#训练函数
def train(dataloader,model,lossfn,optimizer):
    loss,current,n = 0.0,0.0,0
    for batch,(X,y) in enumerate(dataloader):
        #前向传播
        X,y = X.to(device),y.to(device)    #放入GPU
        output = model(X)
        cur_loss = lossfn(output,y)
        _,pred = torch.max(output,axis = 1)
        
        cur_acc = torch.sum(y == pred)/output.shape[0]  #精确度

        #反向传播
        optimizer.zero_grad()
        cur_loss.backward()
        optimizer.step()

        loss += cur_loss.item() #累加loss值
        current += cur_acc.item()#累加精确度
        n = n+1
    print("train_loss"+str(loss/n))
    print("train_acc"+str(current/n))

In [6]:
#验证模型
def val(dataloader,model,lossfn):
    model.eval()#验证模式
    loss,current,n=.0,.0,0

    with torch.no_grad():
        for batch,(X,y) in enumerate(dataloader):
        
            X,y = X.to(device),y.to(device)    #放入GPU
            output = model(X)
            cur_loss = lossfn(output,y)
            _,pred = torch.max(output,axis = 1)
            cur_acc = torch.sum(y == pred)/output.shape[0]  #精确度

            loss += cur_loss.item() #累加loss值
            current += cur_acc.item()#累加精确度
            n = n+1
        # print("以下为验证")
        print("val_loss"+str(loss/n))
        print("val_acc"+str(current/n))

        return current/n

In [7]:
#开始训练
epoch = 10
min_acc = 0
for t in range(epoch):
    print('---------------------\n',f'epoch{t+1}\n---------------------')
    train(train_dataloader,model,lossfn,optimizer)
    a = val(test_dataloader,model,lossfn)

    #做模型更新
    if a > min_acc:
        folder = 'save_model'
        if not os.path.exists(folder):
            os.mkdir('save_model')
        min_acc = a
        print('save best model')
        torch.save(model.state_dict(),'save_model/best_model.pth')
print('Done!')

---------------------
 epoch1
---------------------
train_loss1.484618481940031
train_acc0.44526666666666664
val_loss0.2605002360522747
val_acc0.9227
save best model
---------------------
 epoch2
---------------------
train_loss0.1770688504064145
train_acc0.94455
val_loss0.0996818072680384
val_acc0.9686
save best model
---------------------
 epoch3
---------------------
train_loss0.10471670971640075
train_acc0.9682833333333334
val_loss0.08064643837474286
val_acc0.9745
save best model
---------------------
 epoch4
---------------------
train_loss0.08299302525644889
train_acc0.9745333333333334
val_loss0.06092046292657033
val_acc0.9799
save best model
---------------------
 epoch5
---------------------
train_loss0.072443082660522
train_acc0.9773666666666667
val_loss0.053473096476495265
val_acc0.9841
save best model
---------------------
 epoch6
---------------------
train_loss0.0667228593366143
train_acc0.9793333333333333
val_loss0.05447909789546393
val_acc0.9834
---------------------
 ep

In [8]:
# #测试函数
# import torch
# from torch.autograd import Variable
# from torchvision import datasets,transforms
# from torchvision.transforms import ToPILImage

# model.load_state_dict(torch.load("F:/MyScripts/ipynb/2022暑假动手学深度学习/save_model/best_model.pth"))

# data_transform = transforms.Compose([transforms.ToTensor()])
# test_dataset = datasets.MNIST(root = "./data",train=False,transform=data_transform,download=True)
# test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size = 16,shuffle=True)

# #获取结果
# classes = [
#     "0",
#     "1",
#     "2",
#     "3",
#     "4",
#     "5",
#     "6",
#     "7",
#     "8",
#     "9",
# ]

# #把tensor转化为图片
# show = ToPILImage()

# # 进入验证
# for i in range(5):
#     X,y = test_dataset[i][0],test_dataset[i][1]
#     show(X).show()

#     X = Variable(torch.unsqueeze(X, dim=0).float(), requires_grad=True).to(device)
#     # t = torch.unsqueeze(X, dim=0).float()
#     # X = Variable(t,requires_grad = False)
#     # X.to(device)
#     with torch.no_grad():
#         pred = model(X)

#         predicted,actual = classes[torch.argmax(pred[0])],classes[y]

#         print(f'predicted:"{predicted}",actual:"{actual}"')